In [1]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification

In [2]:
tf.__version__

'2.4.1'

In [3]:
from modeling.multi_output_distilbert import TFDistilBertForFakeNewsClassification

In [4]:
model = TFDistilBertForFakeNewsClassification.from_pretrained('./model', num_labels_aggregate=3, num_labels_category=8)

num aggregate labels:  3
num category labels:  8
All model checkpoint layers were used when initializing TFDistilBertForFakeNewsClassification.

All the layers of TFDistilBertForFakeNewsClassification were initialized from the model checkpoint at ./model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForFakeNewsClassification for predictions without further training.


### start exporting model to one that can be loaded with tensorflow.js

In [5]:
callable = tf.function(model.call)

In [6]:
max_sequence_length = 512

In [7]:
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, max_sequence_length], tf.int32, name="input_ids"), tf.TensorSpec([None, max_sequence_length], tf.int32, name="attention_mask")])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [8]:
print(concrete_function)

ConcreteFunction call(input_ids, attention_mask=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None, return_dict=None, labels=None, training=False)
  Args:
    input_ids: [<1>, <2>]
      <1>: int32 Tensor, shape=(None, 512)
      <2>: int32 Tensor, shape=(None, 512)
  Returns:
    TFFakeNewsOutput([('logits_aggregate', <1>), ('logits_category', <2>)])
      <1>: float32 Tensor, shape=(None, 3)
      <2>: float32 Tensor, shape=(None, 8)


In [10]:
tf.saved_model.save(model, 'distilbert_nela', signatures=concrete_function)

INFO:tensorflow:Assets written to: distilbert_nela\assets
INFO:tensorflow:Assets written to: distilbert_nela\assets


#### run tensorflowjs_converter --input_format=tf_saved_model --saved_model_tags=serve distilbert_nela distilbert_nela_js

In [ ]:
# from tensorflowjs.converters import convert_tf_saved_model

In [11]:
# tests for output sizes
# a = tf.constant(1, shape=[1,3])
# b = tf.constant(1, shape=[1,8])

# def pad_up_to(t, max_in_dims, constant_values):
#     s = tf.shape(t)
#     paddings = [[0, m-s[i]] for (i,m) in enumerate(max_in_dims)]
#     return tf.pad(t, paddings, 'CONSTANT', constant_values=constant_values)

# tf.stack([pad_up_to(a, b.shape, 0), b])
# tf.concat(a, b, 0)

ValueError: Shape (1, 8) must have rank 0

In [1]:
# convert tokenizer vocab
import json
vocab = []
with open('model/vocab.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('##'):  # idk why but the tokenizer library we're using does it the opposite of huggingface
            vocab.append(line.strip('##'))
        elif line.startswith('['):
            vocab.append(line)
        else:
            vocab.append(u'\u2581' + line)

# jsonString = json.dumps(vocab)
# print(jsonString[:50])
with open('../src/detection/tokenizer/vocab.json', 'w+', encoding='utf-8') as f:
    json.dump(vocab, f, ensure_ascii=False)